# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_weather_data = pd.read_csv("output_data/cities.csv")
cities_weather_data.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = cities_weather_data[["Lat", "Lng"]].astype(float)
humidity = cities_weather_data["Humidity"].astype(float)


fig_heat_map = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig_heat_map.add_layer(heat_layer)

fig_heat_map


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
cities_weather_data = cities_weather_data.rename(columns={"Max Temp" : "Max_Temp", "Wind Speed":"Wind_Speed"})

In [5]:
ideal_weather = cities_weather_data.query('35<Humidity<65 & 0<Cloudiness<25 & 60<Max_Temp<90 & 0<Wind_Speed<5')
ideal_weather

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed
86,86,nikolskoye,20,RU,1558378770,56,59.70,30.79,69.01,4.47
153,153,porto novo,20,BR,1558378783,53,-23.68,-45.44,77.00,4.70
223,223,koronowo,20,PL,1558378736,60,53.31,17.94,71.01,3.36
525,525,mecca,1,SA,1558378872,48,21.43,39.83,88.62,1.74


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
from pprint import pprint

In [7]:

# Set base url

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Set parameters for cafes

hotel_parameters = {
    "location": "-23.68, -45.44",
    "keyword": "hotel",
    "radius":50000,
    "type": "hotel",
    "key": g_key
    }

# Request information from Google Places API

hotel_response = requests.get(base_url, params=hotel_parameters)

# Set to json() format

hotel_porto_novo_data = hotel_response.json()

# Pretty print the data
# Removed output since I will be pushing this to github

pprint(hotel_porto_novo_data)

{'html_attributions': [],
 'results': [{'geometry': {'location': {'lat': -23.777735, 'lng': -45.655779},
                           'viewport': {'northeast': {'lat': -23.77634562010727,
                                                      'lng': -45.65443442010727},
                                        'southwest': {'lat': -23.77904527989272,
                                                      'lng': -45.65713407989271}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
              'id': 'f9226ef0a157018e8016bbedf35f229b504ecc05',
              'name': 'Nau Royal',
              'opening_hours': {'open_now': True},
              'photos': [{'height': 480,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.com/maps/contrib/107341097058860204277">Nau '
                                                'Royal</a>'],
                          'photo_reference': 'CmR

In [8]:
hotels_df=pd.DataFrame()
hotels_df["Hotel_Name"]=""
hotels_df["Lat"]=""
hotels_df["Lng"]=""
hotels_df["Address"]=""
hotels_df["Rating"]=""


# Display empty data frame

hotels_df.head()

,Hotel_Name,Lat,Lng,Address,Rating


In [9]:
for i in np.arange(0, len(hotel_porto_novo_data),1):
    hotels_df.loc[i, "Hotel_Name"] = hotel_porto_novo_data["results"][i]["name"]
    hotels_df.loc[i, "Lat"] = hotel_porto_novo_data["results"][i]["geometry"]["location"]["lat"]
    hotels_df.loc[i, "Lng"] = hotel_porto_novo_data["results"][i]["geometry"]["location"]["lng"]
    hotels_df.loc[i, "Address"] = hotel_porto_novo_data["results"][i]["vicinity"]
    hotels_df.loc[i, "Rating"] = hotel_porto_novo_data["results"][i]["rating"]
    
# Display the cafe data frame

hotels_df

,Hotel_Name,Lat,Lng,Address,Rating
0,Nau Royal,-23.7777,-45.6558,"Alameda Patriarca Antonio José Marques, 1533 -...",4.4
1,TW Guaimbê,-23.8547,-45.4136,"Av. Riachuelo, 5360 - Praia do Julião, Ilhabela",4.5
2,Ilha de Toque Toque Boutique Hotel & SPA,-23.8311,-45.5152,"Rod. Dr. Manoel Hipólito do Rêgo, 1285 - Praia...",4.7


In [30]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>Address</dt><dd>{Address}</dd>
<dt>Rating</dt><dd>{Rating}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotels_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

TypeError: unhashable type: 'list'

In [ ]:
# Add marker layer ontop of heat map


# Display Map